In [1]:
# Data를 현재 디렉토리로 다운로드한다.
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-population.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-areas.csv
!curl -O https://raw.githubusercontent.com/jakevdp/data-USstates/master/state-abbrevs.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57935  100 57935    0     0  27614      0  0:00:02  0:00:02 --:--:-- 27614
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   835  100   835    0     0    599      0  0:00:01  0:00:01 --:--:--   599
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   872  100   872    0     0   1427      0 --:--:-- --:--:-- --:--:--  1427


In [2]:
import pandas as pd

# 1. 주어진 데이터를 활용하여 2010년 인구 밀도 기준으로 미국 주와 지역 순위를 계산하라.

## Data loading

In [3]:
population = pd.read_csv('state-population.csv')
areas = pd.read_csv('state-areas.csv')
abbrevs = pd.read_csv('state-abbrevs.csv')

In [4]:
# 인구에 대한 데이터
population.head()

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


In [5]:
# 넓이에 대한 데이터
areas.head()

,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


In [6]:
# 지역 약자에 대한 데이터
abbrevs.head()

,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [7]:
print(population.shape)
print(areas.shape)
print(abbrevs.shape)

(2544, 4)
(52, 2)
(51, 2)


## Data join

Population 데이터에 State full name 데이터를 조인 시키자.

In [8]:
merged = pd.merge(population, abbrevs, how='outer', \
                  left_on='state/region', right_on='abbreviation')

In [9]:
merged.head()

,state/region,ages,year,population,state,abbreviation
0,AL,under18,2012,1117489.0,Alabama,AL
1,AL,total,2012,4817528.0,Alabama,AL
2,AL,under18,2010,1130966.0,Alabama,AL
3,AL,total,2010,4785570.0,Alabama,AL
4,AL,under18,2011,1125763.0,Alabama,AL


Null 값을 체크해보자

In [10]:
merged.isnull().head()

,state/region,ages,year,population,state,abbreviation
0,False,False,False,False,False,False
1,False,False,False,False,False,False
2,False,False,False,False,False,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False


In [11]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
abbreviation     True
dtype: bool

Null값을 확인한다.

In [12]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state,abbreviation
2448,PR,under18,1990,NaN,NaN,NaN
2449,PR,total,1990,NaN,NaN,NaN
2450,PR,total,1991,NaN,NaN,NaN
2451,PR,under18,1991,NaN,NaN,NaN
2452,PR,total,1993,NaN,NaN,NaN


PR 지역의 state 항목(full name)이 누락되어 있음을 볼 수 있다.

Area 정보는 state의 full name을 사용하기 때문에, 누락된 값을 처리해야한다.

State full name 항목이 누락된 곳의 state/region을 살펴보자.

In [13]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

우리의 지식(사전 정보를 이용하면) 누락된 데이터를 쉽게 채워넣을 수 있다.

In [14]:
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico'
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

In [15]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
abbreviation     True
dtype: bool

State full name에 null값이 없음을 확인했다. 따라서 다른 값에 null이 있어도 우선 넘어간다.

다음으로, area 정보가 있는 dataframe을 merge 한다.

In [16]:
final = pd.merge(merged, areas, on='state', how='left')
final.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,AL,52423.0
1,AL,total,2012,4817528.0,Alabama,AL,52423.0
2,AL,under18,2010,1130966.0,Alabama,AL,52423.0
3,AL,total,2010,4785570.0,Alabama,AL,52423.0
4,AL,under18,2011,1125763.0,Alabama,AL,52423.0


Null 값을 다시 확인해본다.

In [17]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
abbreviation      True
area (sq. mi)     True
dtype: bool

Area에 null 값이 존재한다.

In [18]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

United States의 null 값을 다루는 방법
 - Area의 전체 합(미국 전체의 면적이므로)로 값을 채운다.
 - 지역별 인구밀도를 구하는 문제이므로(전체 면적에 대한 정보가 필요없으므로) 그 데이터를 삭제한다.
 
주어진 문제를 해결하기 위해서는 2번째 방법을 선택하면 된다.

In [19]:
final.dropna(inplace=True)

2010 인구밀도를 계산하기 위해, year와 ages를 살펴보자.

In [20]:
final['year'].unique()

array([2012, 2010, 2011, 2009, 2013, 2007, 2008, 2005, 2006, 2004, 2003,
       2001, 2002, 1999, 2000, 1998, 1997, 1996, 1995, 1994, 1993, 1992,
       1991, 1990])

In [21]:
final['ages'].unique()

array(['under18', 'total'], dtype=object)

2010과 전체 나이대에 대한 데이터를 가지고 오겠다.

In [22]:
data_2010 = final.query("year == 2010 & ages == 'total'")
data_2010.head()

,state/region,ages,year,population,state,abbreviation,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,AL,52423.0
91,AK,total,2010,713868.0,Alaska,AK,656425.0
101,AZ,total,2010,6408790.0,Arizona,AZ,114006.0
189,AR,total,2010,2922280.0,Arkansas,AR,53182.0
197,CA,total,2010,37333601.0,California,CA,163707.0


Index를 지정하여 최종결과값이 `Series`가 되어, key가 도시이름이 되도록 한다.

In [23]:
data_2010.set_index('state', inplace=True)
data_2010.head()

,state/region,ages,year,population,abbreviation,area (sq. mi)
state,,,,,,
Alabama,AL,total,2010,4785570.0,AL,52423.0
Alaska,AK,total,2010,713868.0,AK,656425.0
Arizona,AZ,total,2010,6408790.0,AZ,114006.0
Arkansas,AR,total,2010,2922280.0,AR,53182.0
California,CA,total,2010,37333601.0,CA,163707.0


인구 밀도를 계산하자. 

In [24]:
density = data_2010.population / data_2010['area (sq. mi)']
density.head()

state
Alabama        91.287603
Alaska          1.087509
Arizona        56.214497
Arkansas       54.948667
California    228.051342
dtype: float64

인구 밀도 순서대로 정렬하자.

In [25]:
density.sort_values(ascending=False, inplace=True)
density.head()  # 인구밀도가 높은 지역

state
District of Columbia    8898.897059
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
Massachusetts            621.815538
dtype: float64

In [26]:
density.tail()  # 인구밀도가 낮은 지역

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

# 2. (과제) 2007, 2010, 2012 년도의 평균 인구밀도를 구하여라.
$$\text{3개 년도의 평균 인구밀도} = \frac{\frac{1}{3} \times (\text{2007 인구수} + \text{2010 인구수} + \text{2012 인구수})}{\text{면적}}$$